In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
use_gripper = False
if simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml")
elif not simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml") 
elif simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_no_gripper.yaml")
elif not simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_no_gripper.yaml") 
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6004...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


# Run SNOPT IRIS in rational space

In [6]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-4
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
iris_options.q_star = 0.5 * np.zeros(3)
rational_forward_kinematics = RationalForwardKinematics(plant)


#seed points specified in q space
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ]) 
seed_points_t = np.array([rational_forward_kinematics.ComputeTValue(s, iris_options.q_star) for s in seed_points_q])

t_lower = rational_forward_kinematics.ComputeTValue(plant.GetPositionLowerLimits(),iris_options.q_star)
t_upper = rational_forward_kinematics.ComputeTValue(plant.GetPositionUpperLimits(),iris_options.q_star)
P_joint_limits = HPolyhedron.MakeBox(t_lower, t_upper)

In [7]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points_q)}')

Completed region: 1/6
Completed region: 2/6
Completed region: 3/6
Completed region: 4/6
Completed region: 5/6
Completed region: 6/6


In [8]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 50)

# Begin Certification

In [9]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [20]:
editted_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = -.2
binary_search_options.max_iters = 5
binary_search_options.search_d = True

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        C_final, d_final, P_final, q_final = \
            cspace_free_region.CspacePolytopeBilinearAlternation(
                iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
                bilinear_alternation_option, solver_options)
        C_final = np.vstack([C_final, P_joint_limits.A()])
        d_final = np.concatenate([d_final, P_joint_limits.b()])
        editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(editted_regions))

[2022-01-20 21:16:58.565] [console] [info] Solver time 0.03397178649902344
[2022-01-20 21:16:58.743] [console] [info] Solver time 0.04157590866088867
[2022-01-20 21:16:58.904] [console] [info] Solver time 0.03434109687805176
[2022-01-20 21:16:58.905] [console] [info] epsilon=4.9 is infeasible
[2022-01-20 21:16:59.078] [console] [info] Solver time 0.04745292663574219
[2022-01-20 21:16:59.079] [console] [info] epsilon=2.35 is infeasible
[2022-01-20 21:16:59.272] [console] [info] Solver time 0.04851508140563965
[2022-01-20 21:16:59.273] [console] [info] epsilon=1.075 is infeasible
[2022-01-20 21:16:59.448] [console] [info] Solver time 0.05225205421447754
[2022-01-20 21:16:59.450] [console] [info] epsilon=0.4375 is infeasible
[2022-01-20 21:16:59.621] [console] [info] Solver time 0.05344891548156738
[2022-01-20 21:16:59.622] [console] [info] epsilon=0.11875 is infeasible
[2022-01-20 21:16:59.834] [console] [info] Iter: 0, max(log(det(P)))=-0.03320937797731179, solver_time 0.118543863296508

[2022-01-20 21:17:06.419] [console] [info] backoff with log(det(P)) 0.8053514059309257, solver time 0.0849759578704834
[2022-01-20 21:17:06.420] [console] [info] cost improvement 0.006911176612026226
[2022-01-20 21:17:06.525] [console] [info] Iter: 13, polytope step cost -0.4372124874321673, solver time 0.043077945709228516
[2022-01-20 21:17:06.591] [console] [info] back off, cost: -0.43720812044178214, solver time 0.05560493469238281
[2022-01-20 21:17:06.786] [console] [info] Iter: 14, max(log(det(P)))=0.8128949946486901, solver_time 0.10090088844299316
[2022-01-20 21:17:06.899] [console] [info] backoff with log(det(P)) 0.8121543762272465, solver time 0.0979759693145752
[2022-01-20 21:17:06.899] [console] [info] cost improvement 0.006809511397193324
[2022-01-20 21:17:06.997] [console] [info] Iter: 14, polytope step cost -0.448585746166204, solver time 0.035764217376708984
[2022-01-20 21:17:07.104] [console] [info] back off, cost: -0.44858127261623526, solver time 0.09701991081237793
[

[2022-01-20 21:17:13.743] [console] [info] cost improvement 0.003459384718922598
[2022-01-20 21:17:13.834] [console] [info] Iter: 6, polytope step cost -0.3005466823823261, solver time 0.035166025161743164
[2022-01-20 21:17:13.943] [console] [info] back off, cost: -0.30054369288229876, solver time 0.0993509292602539
[2022-01-20 21:17:14.131] [console] [info] Iter: 7, max(log(det(P)))=0.9974867219639894, solver_time 0.10318398475646973
[2022-01-20 21:17:14.233] [console] [info] backoff with log(det(P)) 0.9965826054481102, solver time 0.08720898628234863
[2022-01-20 21:17:14.233] [console] [info] cost improvement 0.0030458235145455204
[2022-01-20 21:17:14.322] [console] [info] Iter: 7, polytope step cost -0.31236749763598665, solver time 0.03260493278503418
[2022-01-20 21:17:14.394] [console] [info] back off, cost: -0.3123643775583738, solver time 0.06203103065490723
[2022-01-20 21:17:14.582] [console] [info] Iter: 8, max(log(det(P)))=1.0003502819953873, solver_time 0.10116291046142578
[

[2022-01-20 21:17:21.022] [console] [info] epsilon=0.4375 is infeasible
[2022-01-20 21:17:21.188] [console] [info] Solver time 0.052021026611328125
[2022-01-20 21:17:21.190] [console] [info] epsilon=0.11875 is infeasible
[2022-01-20 21:17:21.379] [console] [info] Iter: 0, max(log(det(P)))=0.6136718689435359, solver_time 0.1033468246459961
[2022-01-20 21:17:21.484] [console] [info] backoff with log(det(P)) 0.6131108317952594, solver time 0.09176087379455566
[2022-01-20 21:17:21.485] [console] [info] cost improvement inf
[2022-01-20 21:17:21.579] [console] [info] Iter: 0, polytope step cost -0.14655077899523275, solver time 0.037811994552612305
[2022-01-20 21:17:21.653] [console] [info] back off, cost: -0.14654931365612467, solver time 0.06332707405090332
[2022-01-20 21:17:21.853] [console] [info] Iter: 1, max(log(det(P)))=0.8458827261360996, solver_time 0.11524701118469238
[2022-01-20 21:17:21.958] [console] [info] backoff with log(det(P)) 0.845117201236697, solver time 0.09088206291198

[2022-01-20 21:17:27.971] [console] [info] cost improvement 0.0020553438567043614
[2022-01-20 21:17:28.067] [console] [info] Iter: 14, polytope step cost -0.2767016955691024, solver time 0.03962278366088867
[2022-01-20 21:17:28.150] [console] [info] back off, cost: -0.27669898882779176, solver time 0.07258796691894531
[2022-01-20 21:17:28.353] [console] [info] Iter: 15, max(log(det(P)))=1.0124910653066617, solver_time 0.11506795883178711
[2022-01-20 21:17:28.457] [console] [info] backoff with log(det(P)) 1.0115560518290732, solver time 0.0902559757232666
[2022-01-20 21:17:28.458] [console] [info] cost improvement 0.0019470545327655575
[2022-01-20 21:17:28.557] [console] [info] Iter: 15, polytope step cost -0.2760942641529159, solver time 0.042736053466796875
[2022-01-20 21:17:28.672] [console] [info] back off, cost: -0.2760915118172158, solver time 0.10515499114990234
[2022-01-20 21:17:28.879] [console] [info] Iter: 16, max(log(det(P)))=1.0143348885067889, solver_time 0.116979122161865

[2022-01-20 21:17:35.933] [console] [info] Iter: 7, polytope step cost -0.22915032438721292, solver time 0.037847042083740234
[2022-01-20 21:17:36.010] [console] [info] back off, cost: -0.22914803846870646, solver time 0.06580901145935059
[2022-01-20 21:17:36.225] [console] [info] Iter: 8, max(log(det(P)))=0.36613325186883827, solver_time 0.11809802055358887
[2022-01-20 21:17:36.354] [console] [info] backoff with log(det(P)) 0.36580373977493363, solver time 0.11316800117492676
[2022-01-20 21:17:36.355] [console] [info] cost improvement 0.20458419849429146
[2022-01-20 21:17:36.479] [console] [info] Iter: 8, polytope step cost -0.266300381190767, solver time 0.06253504753112793
[2022-01-20 21:17:36.535] [console] [info] back off, cost: -0.26629772327583234, solver time 0.04501199722290039
[2022-01-20 21:17:36.744] [console] [info] Iter: 9, max(log(det(P)))=0.5926508859133586, solver_time 0.1163339614868164
[2022-01-20 21:17:36.864] [console] [info] backoff with log(det(P)) 0.592114006618

[2022-01-20 21:17:43.754] [console] [info] back off, cost: -0.17453809087327046, solver time 0.04614090919494629
[2022-01-20 21:17:43.957] [console] [info] Iter: 6, max(log(det(P)))=0.9888011493557607, solver_time 0.10818886756896973
[2022-01-20 21:17:44.065] [console] [info] backoff with log(det(P)) 0.987908484734328, solver time 0.09252810478210449
[2022-01-20 21:17:44.066] [console] [info] cost improvement 0.004792214741911605
[2022-01-20 21:17:44.194] [console] [info] Iter: 6, polytope step cost -0.1650474612524257, solver time 0.06608986854553223
[2022-01-20 21:17:44.250] [console] [info] back off, cost: -0.16504581211785177, solver time 0.04536294937133789
[2022-01-20 21:17:44.463] [console] [info] Iter: 7, max(log(det(P)))=0.9934827010355977, solver_time 0.11840105056762695
[2022-01-20 21:17:44.573] [console] [info] backoff with log(det(P)) 0.9925839414183031, solver time 0.09541511535644531
[2022-01-20 21:17:44.574] [console] [info] cost improvement 0.0046618284300571355
[2022-

[2022-01-20 21:17:51.693] [console] [info] Solver time 0.03507494926452637
[2022-01-20 21:17:51.694] [console] [info] epsilon=2.35 is infeasible
[2022-01-20 21:17:51.860] [console] [info] Solver time 0.03927206993103027
[2022-01-20 21:17:51.861] [console] [info] epsilon=1.075 is infeasible
[2022-01-20 21:17:52.022] [console] [info] Solver time 0.04402589797973633
[2022-01-20 21:17:52.024] [console] [info] epsilon=0.4375 is infeasible
[2022-01-20 21:17:52.193] [console] [info] Solver time 0.05742502212524414
[2022-01-20 21:17:52.194] [console] [info] epsilon=0.11875 is infeasible
[2022-01-20 21:17:52.369] [console] [info] Iter: 0, max(log(det(P)))=0.5796918670084601, solver_time 0.08030891418457031
[2022-01-20 21:17:52.463] [console] [info] backoff with log(det(P)) 0.579184219036567, solver time 0.0802919864654541
[2022-01-20 21:17:52.464] [console] [info] cost improvement inf
[2022-01-20 21:17:52.558] [console] [info] Iter: 0, polytope step cost -0.1558232818430138, solver time 0.03254

[2022-01-20 21:17:59.326] [console] [info] back off, cost: -0.15190431366528417, solver time 0.06476116180419922
[2022-01-20 21:17:59.530] [console] [info] Iter: 14, max(log(det(P)))=1.0077768431547955, solver_time 0.10814595222473145
[2022-01-20 21:17:59.637] [console] [info] backoff with log(det(P)) 1.0068728532845168, solver time 0.09198188781738281
[2022-01-20 21:17:59.637] [console] [info] cost improvement 0.0024311704336530404
[2022-01-20 21:17:59.740] [console] [info] Iter: 14, polytope step cost -0.15463331245927978, solver time 0.040184974670410156
[2022-01-20 21:17:59.835] [console] [info] back off, cost: -0.1546317775608937, solver time 0.08470010757446289
[2022-01-20 21:18:00.041] [console] [info] Iter: 15, max(log(det(P)))=1.0101266843009342, solver_time 0.10922503471374512
[2022-01-20 21:18:00.150] [console] [info] backoff with log(det(P)) 1.0092192780177167, solver time 0.09442305564880371
[2022-01-20 21:18:00.151] [console] [info] cost improvement 0.0023504493559749484


6


In [21]:
if do_viz:
    visualizer.plot_regions(editted_regions, region_suffix = 'editted')